In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum

In [2]:
spark = SparkSession.builder.appName("Charger_CSV").getOrCreate()

In [12]:
df = spark.read.csv("ventes.csv", header=True, inferSchema=True)

In [13]:
df.show()

+----------+----------+--------+-------------+
|      Date|   Produit|Quantite|Prix_unitaire|
+----------+----------+--------+-------------+
|01/01/2024|Ordinateur|       2|          800|
|01/01/2024|   Clavier|       5|           20|
|02/01/2024|    Souris|      10|           15|
|02/01/2024|Ordinateur|       1|          800|
|03/01/2024|   Clavier|       3|           20|
+----------+----------+--------+-------------+



In [14]:
# Ajout d'une colonne supplémentaire pour le CA
df = df.withColumn("CA", col("Quantite") * col("Prix_unitaire"))

In [15]:
df.show()

+----------+----------+--------+-------------+----+
|      Date|   Produit|Quantite|Prix_unitaire|  CA|
+----------+----------+--------+-------------+----+
|01/01/2024|Ordinateur|       2|          800|1600|
|01/01/2024|   Clavier|       5|           20| 100|
|02/01/2024|    Souris|      10|           15| 150|
|02/01/2024|Ordinateur|       1|          800| 800|
|03/01/2024|   Clavier|       3|           20|  60|
+----------+----------+--------+-------------+----+



In [17]:
# Calcul du CA total 
CA_total = df.agg(_sum("CA")).collect()[0][0]

In [18]:
df.show()

+----------+----------+--------+-------------+----+
|      Date|   Produit|Quantite|Prix_unitaire|  CA|
+----------+----------+--------+-------------+----+
|01/01/2024|Ordinateur|       2|          800|1600|
|01/01/2024|   Clavier|       5|           20| 100|
|02/01/2024|    Souris|      10|           15| 150|
|02/01/2024|Ordinateur|       1|          800| 800|
|03/01/2024|   Clavier|       3|           20|  60|
+----------+----------+--------+-------------+----+



In [19]:
# Produit le plus vendu en quantité
most_sold = df.groupBy("Produit").agg(_sum("Quantite").alias("QuantiteTotale")) 
most_sold = most_sold.orderBy(col("QuantiteTotale").desc()).first()

In [20]:
print(f"CA Total --> : {CA_total}")
print (f"Produit le plus vendu --> : {most_sold}")

CA Total --> : 2710
Produit le plus vendu --> : Row(Produit='Souris', QuantiteTotale=10)


In [22]:
spark.stop()